In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import nltk
#nltk.download('stopwords')
from nltk.corpus import stopwords
#nltk.download('wordnet')
from nltk.stem.wordnet import WordNetLemmatizer
import string

from sklearn.feature_extraction.text import CountVectorizer

from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from scipy import stats
from sklearn.metrics import accuracy_score
from sklearn.metrics import auc, roc_curve
from sklearn.metrics import f1_score, classification_report

from sklearn import svm
import gensim.models as g
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.metrics import auc, roc_curve, roc_auc_score


In [2]:
data = pd.read_csv('/Users/daviddeng8/Documents/Projects/affinity/data/preprocessed_cleaned_body.csv')

In [3]:
right = data.loc[data['Bias'] >= 5]
neutral = data.loc[(data['Bias'] < 5) & (data['Bias'] > -5)]
left = data.loc[data['Bias'] <= -5]
print (len(right) + len(neutral) + len(left))

1675


In [4]:
right['Bias'] = 1
neutral['Bias'] = 0
left['Bias'] = -1

/Users/daviddeng8/anaconda2/envs/practice/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/daviddeng8/anaconda2/envs/practice/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/daviddeng8/anaconda2/envs/practice/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFra

In [5]:
cat_df = pd.concat([right, neutral, left])
cat_df

,Body,Bias,cleaned_body
13,President Donald Trump has vetoed a resolution...,1,president donald trump ha vetoed resolution st...
155,Asked by if he believes President Trump's emer...,1,asked belief president trump emergency declara...
156,You May Like\nThese 10 Shawshank Redemption Fa...,1,may like shawshank redemption fact insane hive...
157,"NEW: This Germantown, Md. man tells he was att...",1,new germantown md man tell wa attacked saturda...
158,“A little as two to three milligrams of fentan...,1,“a little two three milligram fentanyl induce ...
...,...,...,...
1655,"In any sane reality, last week's vote to hold ...",-1,sane reality last week vote hold new election ...
1656,People have expressed concern that Lindsey Gra...,-1,people expressed concern lindsey graham shamel...
1657,THE DAY OF THE BIGGEST COVER-UP OF ALL IS NIGH...,-1,day biggest coverup nigh thursday anticipating...
1658,"If you are a man or a lady of letters, and are...",-1,man lady letter also versed internet tradition...


In [6]:
vectorizer = TfidfVectorizer(max_features=800)
tfidf = vectorizer.fit_transform(cat_df['cleaned_body'])
X = pd.DataFrame(tfidf.toarray(), columns=vectorizer.get_feature_names())
y = cat_df['Bias']

In [7]:
y

13      1
155     1
156     1
157     1
158     1
       ..
1655   -1
1656   -1
1657   -1
1658   -1
1659   -1
Name: Bias, Length: 1675, dtype: int64

In [8]:
col_names = X.columns

scaler = MinMaxScaler()
scaled = scaler.fit_transform(X)
X = pd.DataFrame(scaled, columns=col_names)

In [9]:
X

,ability,able,abortion,abuse,access,according,account,accused,across,act,...,would,wrong,wrote,year,yearold,yet,york,you,young,zealand
0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.171555,...,0.000000,0.000000,0.000000,0.135738,0.0,0.0,0.000000,0.0,0.0,0.0
1,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0
2,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0
3,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0
4,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1670,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.238936,0.000000,0.172692,0.063994,0.0,0.0,0.000000,0.0,0.0,0.0
1671,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.072003,0.587958,0.000000,0.057853,0.0,0.0,0.130847,0.0,0.0,0.0
1672,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.216877,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0
1673,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.334580,0.455350,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0


In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=3)

pandas.core.frame.DataFrame

In [11]:
model = svm.SVC(probability = True)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("Accuracy score: ", accuracy_score(y_test, y_pred))

Accuracy score:  0.6014319809069213


In [12]:
y_probs = model.predict_proba(X_test)
roc_auc = roc_auc_score(y_test, y_probs, average="macro", multi_class="ovo")
print('ROC Score is: ', roc_auc)

ROC Score is:  0.7844348180207481


In [13]:
right

,Body,Bias,cleaned_body
13,President Donald Trump has vetoed a resolution...,1,president donald trump ha vetoed resolution st...
155,Asked by if he believes President Trump's emer...,1,asked belief president trump emergency declara...
156,You May Like\nThese 10 Shawshank Redemption Fa...,1,may like shawshank redemption fact insane hive...
157,"NEW: This Germantown, Md. man tells he was att...",1,new germantown md man tell wa attacked saturda...
158,“A little as two to three milligrams of fentan...,1,“a little two three milligram fentanyl induce ...
...,...,...,...
1670,Only hours after the first flames that eventua...,1,hour first flame eventually would consume notr...
1671,Washington watchdog Judicial Watch believes th...,1,washington watchdog judicial watch belief fbi ...
1672,"Two Fox News personalities, as Notre Dame Cath...",1,two fox news personality notre dame cathedral ...
1673,Chase Bank is shutting down accounts of people...,1,chase bank shutting account people organizatio...


In [14]:
neutral

,Body,Bias,cleaned_body
0,Abortion rights advocates have asked the U.S. ...,0,abortion right advocate asked u supreme court ...
1,A federal appeals court rejected the most dire...,0,federal appeal court rejected direct constitut...
2,As part of the Trump administration's effort t...,0,part trump administration effort slow migrant ...
3,"President Donald Trump and ""the Trump of the T...",0,president donald trump trump tropic brazilian ...
5,Facebook announced Tuesday that it will block ...,0,facebook announced tuesday block landlord cred...
...,...,...,...
1633,"In that previous case, the town of Greece, in ...",0,previous case town greece upstate new york ope...
1636,This combination of undated photos released by...,0,combination undated photo released jefferson c...
1641,The prototypes built as part of President Trum...,0,prototype built part president trump’s border ...
1661,"(PASTEMAGAZINE) — Monday night, Bernie Sanders...",0,pastemagazine — monday night bernie sander tow...


In [15]:
left

,Body,Bias,cleaned_body
4,"U.S Senator Elizabeth Warren, who is competing...",-1,u senator elizabeth warren competing democrati...
10,Presidential candidates submitted their first ...,-1,presidential candidate submitted first quarter...
11,Even as President Donald Trump continues his f...,-1,even president donald trump continues feud rep...
12,President Donald Trump again criticized the la...,-1,president donald trump criticized late sen joh...
14,U.S. cyber operators disrupted internet access...,-1,u cyber operator disrupted internet access pur...
...,...,...,...
1655,"In any sane reality, last week's vote to hold ...",-1,sane reality last week vote hold new election ...
1656,People have expressed concern that Lindsey Gra...,-1,people expressed concern lindsey graham shamel...
1657,THE DAY OF THE BIGGEST COVER-UP OF ALL IS NIGH...,-1,day biggest coverup nigh thursday anticipating...
1658,"If you are a man or a lady of letters, and are...",-1,man lady letter also versed internet tradition...


In [16]:
type(X_train)

pandas.core.frame.DataFrame

### train with kfold cross validation

In [ ]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=2)
